In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install -Uqq pandas investpy python-dateutil networkx matplotlib scikit-learn great-expectations seaborn

In [4]:
import os
import re
import time
import pandas as pd
import matplotlib.pyplot as plt
import investpy
from pathlib import Path
from dateutil.relativedelta import relativedelta
from dataclasses import dataclass, field
from datetime import date
from typing import List

In [5]:
%matplotlib inline
pd.set_option('display.max_rows', 500)

In [6]:
# Global variables and constants
COUNTRY = "malaysia"                   # use investpy.get_stock_countries() to list supported countries
DATASET_FOLDER = "test_data"
TARGET_DATASET_FOLDER = f"{DATASET_FOLDER}/{COUNTRY}"
STOCKS_DATASET = f"{TARGET_DATASET_FOLDER}/stocks.csv"
STOCKS_INFO_DATASET = f"{TARGET_DATASET_FOLDER}/stocks_info.csv"
STOCKS_FINANCE_DATASET = f"{TARGET_DATASET_FOLDER}/stocks_finance.csv"
STOCKS_DIVIDENDS_DATASET = f"{TARGET_DATASET_FOLDER}/stocks_dividends.csv"
STOCKS_SELECTED = f"{TARGET_DATASET_FOLDER}/stocks_selected.csv"

In [7]:
def create_folder(folder):
    """Create folder if not exists"""
    if not os.path.exists(folder):
        os.makedirs(folder)

@dataclass
class DividendYield:
    year:int = 0
    percentage: float = 0.0
    
@dataclass
class Stock:
    country: str = ''
    name: str = ''
    symbol: str = ''
    pe_ratio: float = 0.0
    yields: List[int] = field(default_factory=list)

# Create target folder
create_folder(TARGET_DATASET_FOLDER)

In [8]:
# Get stocks for the country
df_stocks = investpy.get_stocks(country=COUNTRY)
df_stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    929 non-null    object
 1   name       929 non-null    object
 2   full_name  929 non-null    object
 3   isin       929 non-null    object
 4   currency   929 non-null    object
 5   symbol     929 non-null    object
dtypes: object(6)
memory usage: 43.7+ KB


In [9]:
display(df_stocks.head(10))

,country,name,full_name,isin,currency,symbol
0,malaysia,Media Chinese Int,Media Chinese International Ltd,BMG5959D1048,MYR,MDCH
1,malaysia,AMMB,AMMB Holdings Bhd,MYL1015OO006,MYR,AMMB
2,malaysia,CIMB Group,CIMB Group Holdings Bhd,MYL1023OO000,MYR,CIMB
3,malaysia,RHB Bank,RHB Bank Bhd,MYL1066OO009,MYR,RHBC
4,malaysia,Hong Leong Financial,Hong Leong Financial Group Bhd,MYL1082OO006,MYR,HLCB
5,malaysia,Malayan Banking,Malayan Banking Bhd,MYL1155OO000,MYR,MBBM
6,malaysia,Malaysia Building,Malaysia Building Society Bhd,MYL1171OO007,MYR,MBSS
7,malaysia,Public Bank,Public Bank Bhd,MYL1295OO004,MYR,PUBM
8,malaysia,Berjaya Sports Toto,Berjaya Sports Toto Bhd,MYL1562OO007,MYR,BSTB
9,malaysia,DRB Hicom,DRB - Hicom Bhd,MYL1619OO005,MYR,DRBM


In [10]:
def save_csv(df, file_name):
    df.to_csv(file_name, header=True, index=False)

# Save stock list
save_csv(df_stocks, STOCKS_DATASET)

In [11]:
def get_stock_info(symbol, country):
    try:
        return investpy.get_stock_information(symbol, country)
    except:
        return None
    
def get_stock_dividends(symbol, country):
    try:
        return investpy.get_stock_dividends(symbol, country)
    except:
        return None
    
def read_csv(file):
    if not os.path.isfile(file):
        return None
    return pd.read_csv(file)

def download_stocks_info(df):
    df_stocks_info = None
    count = 0
    for _, row in df.iterrows():
        count = count + 1
        print(f"{count}/{len(df)}: {row.symbol}-{row['name']}")
        df_stock = get_stock_info(row.symbol, row.country)
        if df_stock is None:
            continue
        if df_stocks_info is None:
            df_stocks_info  = df_stock
        else:
            df_stocks_info = df_stocks_info.append(df_stock)
        if count % 10 == 0:
            save_csv(df_stocks_info, STOCKS_INFO_DATASET)
            time.sleep(3)
    save_csv(df_stocks_info, STOCKS_INFO_DATASET)

def download_stocks_dividends(df):
    df_stocks_dividends = None
    count = 0
    for _, row in df.iterrows():
            count = count + 1
            print(f"{count}/{len(df)}: {row.symbol}-{row['name']}")
            df_stock = get_stock_dividends(row.symbol, row.country)
            if df_stock is None:
                continue
            df_stock['Symbol'] = row.symbol
            if df_stocks_dividends is None:
                df_stocks_dividends  = df_stock
            else:
                df_stocks_dividends = df_stocks_dividends.append(df_stock)
            if count % 10 == 0:
                save_csv(df_stocks_dividends, STOCKS_DIVIDENDS_DATASET)
                time.sleep(3)
    save_csv(df_stocks_dividends, STOCKS_DIVIDENDS_DATASET)

In [12]:
#download_stocks_info(df_stocks)

#download_stocks_dividends(df_stocks)

In [13]:
df_stocks_info = read_csv(STOCKS_INFO_DATASET)
display(df_stocks_info.head(10))

# Convert PE ratio
df_stocks_info['PE'] = df_stocks_info['P/E Ratio'].apply(lambda x: float(x.replace('%','').replace('-','0')))
#df_stocks_info['Next Earning'] = pd.to_datetime(df_stocks_info['Next Earnings Date'], format='%d/%m/%Y', errors='ignore')
display(df_stocks_info.head(10))
df_stocks_info.info()

,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date
0,MDCH,0.19,0.19-0.195,490.27M,0.19,0.14-0.235,-,498000.0,311.1M,0.00(2.16%),5485763.0,-,-,26.67%,1.687236e+09,01/09/2021
1,AMMB,2.90,2.9-3.02,3900000000.0,2.91,2.63-3.79,-1.27,9332600.0,9610000000.0,N/A(N/A),3181045.0,-,1.1,-2.68%,3.314011e+09,26/08/2021
2,CIMB,4.68,4.68-4.83,16420000000.000002,4.68,2.9-4.83,0.32,26748800.0,46870000000.0,0.05(1.03%),11481116.0,14.77,1.95,36.44%,1.001418e+10,30/08/2021
3,RHBC,5.34,5.35-5.43,5380000000.0,5.36,4.17-5.98,0.52,6028500.0,21730000000.0,0.18(3.37%),2387024.0,10.31,1.01,11.25%,4.068360e+09,01/09/2021
4,HLCB,17.32,17.3-18.02,4680000000.0,17.48,12.34-18.7,1.96,289700.0,19640000000.0,0.36(2.07%),166766.0,8.86,1.04,37.68%,1.133789e+09,01/09/2021
5,MBBM,8.17,8.16-8.32,22890000000.0,8.17,6.96-9.01,0.6,11938400.0,95530000000.0,0.52(6.50%),6173324.0,13.53,1.07,8.21%,1.169334e+10,26/08/2021
6,MBSS,0.61,0.61-0.635,1640000000.0,0.61,0.485-0.77,0.06,18132000.0,4370000000.0,0.02(3.33%),6570521.0,10.23,1.16,11.93%,7.171484e+09,02/09/2021
7,PUBM,3.94,3.93-4.13,9480000000.0,3.94,2.97-4.97,0.26,56126400.0,76480000000.0,0.03(3.30%),17805673.0,15.08,0.86,13.87%,1.941069e+10,24/08/2021
8,BSTB,2.00,1.99-2.04,4830000000.0,2.00,1.88-2.27,0.14,811000.0,2680000000.0,0.12(6.16%),577756.0,14.73,0.08,-4.27%,1.341900e+09,16/11/2021
9,DRBM,1.59,1.6-1.65,14550000000.0,1.60,1.58-2.25,0.41,2419300.0,3070000000.0,0.02(1.24%),1283879.0,3.85,2.26,-22.82%,1.933237e+09,26/08/2021


,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date,PE
0,MDCH,0.19,0.19-0.195,490.27M,0.19,0.14-0.235,-,498000.0,311.1M,0.00(2.16%),5485763.0,-,-,26.67%,1.687236e+09,01/09/2021,0.00
1,AMMB,2.90,2.9-3.02,3900000000.0,2.91,2.63-3.79,-1.27,9332600.0,9610000000.0,N/A(N/A),3181045.0,-,1.1,-2.68%,3.314011e+09,26/08/2021,0.00
2,CIMB,4.68,4.68-4.83,16420000000.000002,4.68,2.9-4.83,0.32,26748800.0,46870000000.0,0.05(1.03%),11481116.0,14.77,1.95,36.44%,1.001418e+10,30/08/2021,14.77
3,RHBC,5.34,5.35-5.43,5380000000.0,5.36,4.17-5.98,0.52,6028500.0,21730000000.0,0.18(3.37%),2387024.0,10.31,1.01,11.25%,4.068360e+09,01/09/2021,10.31
4,HLCB,17.32,17.3-18.02,4680000000.0,17.48,12.34-18.7,1.96,289700.0,19640000000.0,0.36(2.07%),166766.0,8.86,1.04,37.68%,1.133789e+09,01/09/2021,8.86
5,MBBM,8.17,8.16-8.32,22890000000.0,8.17,6.96-9.01,0.6,11938400.0,95530000000.0,0.52(6.50%),6173324.0,13.53,1.07,8.21%,1.169334e+10,26/08/2021,13.53
6,MBSS,0.61,0.61-0.635,1640000000.0,0.61,0.485-0.77,0.06,18132000.0,4370000000.0,0.02(3.33%),6570521.0,10.23,1.16,11.93%,7.171484e+09,02/09/2021,10.23
7,PUBM,3.94,3.93-4.13,9480000000.0,3.94,2.97-4.97,0.26,56126400.0,76480000000.0,0.03(3.30%),17805673.0,15.08,0.86,13.87%,1.941069e+10,24/08/2021,15.08
8,BSTB,2.00,1.99-2.04,4830000000.0,2.00,1.88-2.27,0.14,811000.0,2680000000.0,0.12(6.16%),577756.0,14.73,0.08,-4.27%,1.341900e+09,16/11/2021,14.73
9,DRBM,1.59,1.6-1.65,14550000000.0,1.60,1.58-2.25,0.41,2419300.0,3070000000.0,0.02(1.24%),1283879.0,3.85,2.26,-22.82%,1.933237e+09,26/08/2021,3.85


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Stock Symbol        923 non-null    object 
 1   Prev. Close         923 non-null    float64
 2   Todays Range        923 non-null    object 
 3   Revenue             923 non-null    object 
 4   Open                923 non-null    float64
 5   52 wk Range         923 non-null    object 
 6   EPS                 923 non-null    object 
 7   Volume              923 non-null    float64
 8   Market Cap          923 non-null    object 
 9   Dividend (Yield)    923 non-null    object 
 10  Average Vol. (3m)   923 non-null    object 
 11  P/E Ratio           923 non-null    object 
 12  Beta                923 non-null    object 
 13  1-Year Change       923 non-null    object 
 14  Shares Outstanding  923 non-null    float64
 15  Next Earnings Date  923 non-null    object 
 16  PE      

In [14]:
df_stocks_dividends = read_csv(STOCKS_DIVIDENDS_DATASET)
display(df_stocks_dividends.head(10))
#display(df_stocks_dividends[df_stocks_dividends['Symbol'] == 'MBBM'].head(10))

df_stocks_dividends['Year'] = df_stocks_dividends['Date'].apply(lambda x: int(x[0:4]))
df_stocks_dividends['Yield'] = df_stocks_dividends['Yield'].apply(lambda x: float(x.replace('%','').replace('-','0')))
display(df_stocks_dividends[df_stocks_dividends['Symbol'] == 'MBBM'].head(10))
df_stocks_dividends.info()

,Date,Dividend,Type,Payment Date,Yield,Symbol
0,2021-06-18,0.0041,trailing_twelve_months,2021-07-08,2.16%,MDCH
1,2020-07-10,0.0041,trailing_twelve_months,2020-07-30,2.28%,MDCH
2,2019-12-09,0.0109,trailing_twelve_months,2019-12-30,5.59%,MDCH
3,2019-06-19,0.0117,trailing_twelve_months,2019-07-12,7.31%,MDCH
4,2018-12-12,0.0151,trailing_twelve_months,2018-12-28,7.95%,MDCH
5,2018-06-18,0.0171,trailing_twelve_months,2018-07-13,8.55%,MDCH
6,2017-12-11,0.0250,trailing_twelve_months,2017-12-29,8.47%,MDCH
7,2017-06-16,0.0307,trailing_twelve_months,2017-07-10,7.31%,MDCH
8,2016-12-13,0.0428,trailing_twelve_months,2016-12-30,7.25%,MDCH
9,2016-06-20,0.0451,trailing_twelve_months,2016-07-13,6.35%,MDCH


,Date,Dividend,Type,Payment Date,Yield,Symbol,Year
58,2021-05-28,0.520,trailing_twelve_months,2021-06-28,6.50,MBBM,2021
59,2020-12-17,0.525,trailing_twelve_months,2021-01-15,6.41,MBBM,2020
60,2020-04-10,0.640,trailing_twelve_months,2020-05-06,7.71,MBBM,2020
61,2019-09-18,0.570,trailing_twelve_months,2019-10-02,7.84,MBBM,2019
62,2019-05-07,0.570,trailing_twelve_months,2019-06-07,6.56,MBBM,2019
63,2018-09-28,0.570,trailing_twelve_months,2018-10-30,6.23,MBBM,2018
64,2018-06-05,0.550,trailing_twelve_months,2018-07-06,5.56,MBBM,2018
65,2017-09-29,0.550,trailing_twelve_months,2017-11-01,5.06,MBBM,2017
66,2017-05-04,0.520,trailing_twelve_months,2017-06-06,5.36,MBBM,2017
67,2016-09-23,0.500,trailing_twelve_months,2016-10-25,5.52,MBBM,2016


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4674 entries, 0 to 4673
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          4674 non-null   object 
 1   Dividend      4674 non-null   float64
 2   Type          4674 non-null   object 
 3   Payment Date  4674 non-null   object 
 4   Yield         4674 non-null   float64
 5   Symbol        4674 non-null   object 
 6   Year          4674 non-null   int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 255.7+ KB


In [15]:
# Average 2 years yield
df_dividends_summary = df_stocks_dividends[ (df_stocks_dividends['Year'] == 2021) | (df_stocks_dividends['Year'] == 2020) ][['Symbol','Yield']]

display(df_dividends_summary[df_dividends_summary['Symbol']=='MBBM'].head(10))

df_dividends_summary = df_dividends_summary.groupby(['Symbol']).sum()
df_dividends_summary['Yield'] = df_dividends_summary['Yield'] / 2   # Average over 2 years
# df_dividends_summary = df_dividends_summary.groupby(['Symbol']).mean()

df_dividends_summary.reset_index(inplace=True)

display(df_dividends_summary.head(20))

,Symbol,Yield
58,MBBM,6.50
59,MBBM,6.41
60,MBBM,7.71


,Symbol,Yield
0,ADVA,1.600
1,ADVN,1.515
2,AEOM,2.280
3,AFHB,2.565
4,AFIN,3.090
5,AHBM,6.430
6,AINM,4.100
7,AJIN,1.575
8,ALLI,1.230
9,ALSA,5.800


In [16]:
# https://ng.investing.com/equities/malayan-banking-bhd-ratios

df_summary = pd.merge(df_stocks_info, df_dividends_summary, how='left', left_on=['Stock Symbol'], right_on=['Symbol'])
display(df_summary.head(10))

# Choose columns
df_summary = df_summary[['Symbol', 'PE', 'Yield']]
display(df_summary.head(10))

,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date,PE,Symbol,Yield
0,MDCH,0.19,0.19-0.195,490.27M,0.19,0.14-0.235,-,498000.0,311.1M,0.00(2.16%),5485763.0,-,-,26.67%,1.687236e+09,01/09/2021,0.00,MDCH,2.220
1,AMMB,2.90,2.9-3.02,3900000000.0,2.91,2.63-3.79,-1.27,9332600.0,9610000000.0,N/A(N/A),3181045.0,-,1.1,-2.68%,3.314011e+09,26/08/2021,0.00,AMMB,2.350
2,CIMB,4.68,4.68-4.83,16420000000.000002,4.68,2.9-4.83,0.32,26748800.0,46870000000.0,0.05(1.03%),11481116.0,14.77,1.95,36.44%,1.001418e+10,30/08/2021,14.77,CIMB,3.330
3,RHBC,5.34,5.35-5.43,5380000000.0,5.36,4.17-5.98,0.52,6028500.0,21730000000.0,0.18(3.37%),2387024.0,10.31,1.01,11.25%,4.068360e+09,01/09/2021,10.31,RHBC,6.350
4,HLCB,17.32,17.3-18.02,4680000000.0,17.48,12.34-18.7,1.96,289700.0,19640000000.0,0.36(2.07%),166766.0,8.86,1.04,37.68%,1.133789e+09,01/09/2021,8.86,HLCB,2.215
5,MBBM,8.17,8.16-8.32,22890000000.0,8.17,6.96-9.01,0.6,11938400.0,95530000000.0,0.52(6.50%),6173324.0,13.53,1.07,8.21%,1.169334e+10,26/08/2021,13.53,MBBM,10.310
6,MBSS,0.61,0.61-0.635,1640000000.0,0.61,0.485-0.77,0.06,18132000.0,4370000000.0,0.02(3.33%),6570521.0,10.23,1.16,11.93%,7.171484e+09,02/09/2021,10.23,MBSS,4.065
7,PUBM,3.94,3.93-4.13,9480000000.0,3.94,2.97-4.97,0.26,56126400.0,76480000000.0,0.03(3.30%),17805673.0,15.08,0.86,13.87%,1.941069e+10,24/08/2021,15.08,PUBM,3.430
8,BSTB,2.00,1.99-2.04,4830000000.0,2.00,1.88-2.27,0.14,811000.0,2680000000.0,0.12(6.16%),577756.0,14.73,0.08,-4.27%,1.341900e+09,16/11/2021,14.73,BSTB,14.250
9,DRBM,1.59,1.6-1.65,14550000000.0,1.60,1.58-2.25,0.41,2419300.0,3070000000.0,0.02(1.24%),1283879.0,3.85,2.26,-22.82%,1.933237e+09,26/08/2021,3.85,DRBM,1.150


,Symbol,PE,Yield
0,MDCH,0.00,2.220
1,AMMB,0.00,2.350
2,CIMB,14.77,3.330
3,RHBC,10.31,6.350
4,HLCB,8.86,2.215
5,MBBM,13.53,10.310
6,MBSS,10.23,4.065
7,PUBM,15.08,3.430
8,BSTB,14.73,14.250
9,DRBM,3.85,1.150


In [17]:
# Short list stocks
df_selected = df_summary[  (df_summary['PE'] < 15) & (df_summary['PE'] > 0) & (df_summary['Yield'] > 8) ]

display(df_selected.head(100))

df_temp = df_stocks_info[ df_stocks_info['Stock Symbol'].isin(df_selected['Symbol']) ]
display(df_temp)
# TODO - earning date is < 30 days from now
display( df_temp[ pd.to_datetime(df_temp['Next Earnings Date'], format='%d/%m/%Y', errors='coerce') > pd.Timestamp('today')] )

display(df_stocks[ df_stocks['symbol'].isin(df_selected['Symbol']) ])

display(df_stocks_dividends[ df_stocks_dividends['Symbol'].isin(df_selected['Symbol']) ])

# Save
save_csv(df_selected, STOCKS_SELECTED)

,Symbol,PE,Yield
5,MBBM,13.53,10.310
8,BSTB,14.73,14.250
11,BMYS,13.57,8.620
31,BATO,14.74,31.240
42,UOAD,10.75,8.320
46,TPGC,3.58,17.435
82,ASTR,9.37,21.245
88,KRIB,2.61,8.055
155,BERA,13.74,12.810
231,DOMN,8.98,10.835


,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date,PE
5,MBBM,8.170,8.16-8.32,22890000000.0,8.170,6.96-9.01,0.6,11938400.0,95530000000.0,0.52(6.50%),6173324.0,13.53,1.07,8.21%,1.169334e+10,26/08/2021,13.53
8,BSTB,2.000,1.99-2.04,4830000000.0,2.000,1.88-2.27,0.14,811000.0,2680000000.0,0.12(6.16%),577756.0,14.73,0.08,-4.27%,1.341900e+09,16/11/2021,14.73
11,BMYS,7.330,7.35-7.53,878.24M,7.350,7.31-9.9,0.54,3397500.0,5930000000.0,0.50(6.61%),1280013.0,13.57,1.21,-26.26%,8.092991e+08,02/11/2021,13.57
31,BATO,14.000,13.96-14.1,2450000000.0,13.980,9.8-16.2,0.95,178200.0,359310000000.0,0.93(6.60%),316373.0,14.74,0.49,35.92%,2.288578e+09,24/08/2021,14.74
42,UOAD,1.590,1.58-1.62,609.5M,1.600,1.46-1.92,0.15,1813800.0,3700000000.0,0.14(8.86%),558834.0,10.75,0.78,-4.22%,2.326466e+09,31/08/2021,10.75
46,TPGC,3.760,3.74-3.85,17400000000.0,3.770,3.64-9.72,1.05,7641600.0,30110000000.0,0.59(15.72%),19801147.0,3.58,0.78,-59.7%,8.006863e+09,17/09/2021,3.58
82,ASTR,1.090,1.08-1.11,4370000000.0,1.090,0.71-1.26,0.12,3623600.0,5680000000.0,0.09(7.87%),5383905.0,9.37,1.32,42.48%,5.214507e+09,16/09/2021,9.37
88,KRIB,3.060,3.04-3.17,6760000000.0,3.070,2.93-8.39,1.17,3109300.0,7810000000.0,0.24(7.64%),5491692.0,2.61,1.42,-62.68%,2.551620e+09,08/11/2021,2.61
155,BERA,1.580,1.57-1.63,2290000000.0,1.590,1.12-1.64,0.12,4542600.0,1840000000.0,0.05(3.17%),1953342.0,13.74,1.35,14.49%,1.161428e+09,16/09/2021,13.74
231,DOMN,0.815,0.825-0.87,601.04M,0.825,0.655-1.2,0.09,589800.0,134.67M,0.03(4.29%),84497.0,8.98,0.49,18.12%,1.652402e+08,30/08/2021,8.98


,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date,PE
8,BSTB,2.000,1.99-2.04,4830000000.0,2.000,1.88-2.27,0.14,811000.0,2680000000.0,0.12(6.16%),577756.0,14.73,0.08,-4.27%,1.341900e+09,16/11/2021,14.73
11,BMYS,7.330,7.35-7.53,878.24M,7.350,7.31-9.9,0.54,3397500.0,5930000000.0,0.50(6.61%),1280013.0,13.57,1.21,-26.26%,8.092991e+08,02/11/2021,13.57
42,UOAD,1.590,1.58-1.62,609.5M,1.600,1.46-1.92,0.15,1813800.0,3700000000.0,0.14(8.86%),558834.0,10.75,0.78,-4.22%,2.326466e+09,31/08/2021,10.75
46,TPGC,3.760,3.74-3.85,17400000000.0,3.770,3.64-9.72,1.05,7641600.0,30110000000.0,0.59(15.72%),19801147.0,3.58,0.78,-59.7%,8.006863e+09,17/09/2021,3.58
82,ASTR,1.090,1.08-1.11,4370000000.0,1.090,0.71-1.26,0.12,3623600.0,5680000000.0,0.09(7.87%),5383905.0,9.37,1.32,42.48%,5.214507e+09,16/09/2021,9.37
88,KRIB,3.060,3.04-3.17,6760000000.0,3.070,2.93-8.39,1.17,3109300.0,7810000000.0,0.24(7.64%),5491692.0,2.61,1.42,-62.68%,2.551620e+09,08/11/2021,2.61
155,BERA,1.580,1.57-1.63,2290000000.0,1.590,1.12-1.64,0.12,4542600.0,1840000000.0,0.05(3.17%),1953342.0,13.74,1.35,14.49%,1.161428e+09,16/09/2021,13.74
231,DOMN,0.815,0.825-0.87,601.04M,0.825,0.655-1.2,0.09,589800.0,134.67M,0.03(4.29%),84497.0,8.98,0.49,18.12%,1.652402e+08,30/08/2021,8.98
321,HLIB,9.190,9.16-9.24,2340000000.0,9.190,7.35-10.42,0.71,7600.0,2890000000.0,0.35(3.85%),51495.0,12.92,0.8,19.66%,3.142348e+08,01/09/2021,12.92
359,JHTN,1.440,1.43-1.47,507.9M,1.450,1.42-2.08,0.14,1406700.0,442.89M,0.06(3.88%),432769.0,9.98,0.75,6.67%,3.075628e+08,01/09/2021,9.98


,country,name,full_name,isin,currency,symbol
5,malaysia,Malayan Banking,Malayan Banking Bhd,MYL1155OO000,MYR,MBBM
8,malaysia,Berjaya Sports Toto,Berjaya Sports Toto Bhd,MYL1562OO007,MYR,BSTB
11,malaysia,Bursa Malaysia,Bursa Malaysia Bhd,MYL1818OO003,MYR,BMYS
31,malaysia,BAT Malaysia,British American Tobacco Malaysia Bhd,MYL4162OO003,MYR,BATO
42,malaysia,UOA Development,UOA Development Bhd,MYL5200OO000,MYR,UOAD
46,malaysia,Top Glove Corp,Top Glove Corporation Bhd,MYL7113OO003,MYR,TPGC
84,malaysia,Astro Malaysia,Astro Malaysia Holdings Bhd,MYL6399OO009,MYR,ASTR
90,malaysia,Kossan Rubber Ind,Kossan Rubber Industries Bhd,MYL7153OO009,MYR,KRIB
157,malaysia,Bermaz Auto,Bermaz Auto Bhd,MYL5248OO009,MYR,BERA
233,malaysia,Dominant Enterprise,Dominant Enterprise Bhd,MYL7169OO005,MYR,DOMN


,Date,Dividend,Type,Payment Date,Yield,Symbol,Year
58,2021-05-28,0.5200,trailing_twelve_months,2021-06-28,6.50,MBBM,2021
59,2020-12-17,0.5250,trailing_twelve_months,2021-01-15,6.41,MBBM,2020
60,2020-04-10,0.6400,trailing_twelve_months,2020-05-06,7.71,MBBM,2020
61,2019-09-18,0.5700,trailing_twelve_months,2019-10-02,7.84,MBBM,2019
62,2019-05-07,0.5700,trailing_twelve_months,2019-06-07,6.56,MBBM,2019
63,2018-09-28,0.5700,trailing_twelve_months,2018-10-30,6.23,MBBM,2018
64,2018-06-05,0.5500,trailing_twelve_months,2018-07-06,5.56,MBBM,2018
65,2017-09-29,0.5500,trailing_twelve_months,2017-11-01,5.06,MBBM,2017
66,2017-05-04,0.5200,trailing_twelve_months,2017-06-06,5.36,MBBM,2017
67,2016-09-23,0.5000,trailing_twelve_months,2016-10-25,5.52,MBBM,2016


## Stock Clustering

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

In [19]:
df_analysis = pd.merge(df_stocks_info, df_dividends_summary, how='left', left_on=['Stock Symbol'], right_on=['Symbol'])
df_analysis['Symbol'] = df_analysis['Symbol'].astype('category')
display(df_analysis.head(10))

# Drop columnss 
cols = ['Next Earnings Date', 'Todays Range', 'Prev. Close', 'Open', '52 wk Range', 'Dividend (Yield)', 'P/E Ratio', 'Stock Symbol']
df_analysis.drop(columns=cols, inplace=True)
display(df_analysis.head(10))

def convert_uom(v): 
    if "M" in v:
        return float(v[0:len(v)-1]) * 1_000_000
    elif "K" in v:
        return float(v[0:len(v)-1]) * 1_000
    elif v == "-":
        return 0
    elif "%" in v:
        return float(v[0:len(v)-1].replace(",",""))
    else:
        return float(v)
    
df_analysis['Revenue'] = df_analysis['Revenue'].apply(convert_uom)
df_analysis['EPS'] = df_analysis['EPS'].apply(convert_uom)
df_analysis['Market Cap'] = df_analysis['Market Cap'].apply(convert_uom)
df_analysis['Average Vol. (3m)'] = df_analysis['Average Vol. (3m)'].apply(convert_uom)
df_analysis['Beta'] = df_analysis['Beta'].apply(convert_uom)
df_analysis['1-Year Change'] = df_analysis['1-Year Change'].apply(convert_uom)

display(df_analysis.head(10))

display(df_analysis.info())

,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date,PE,Symbol,Yield
0,MDCH,0.19,0.19-0.195,490.27M,0.19,0.14-0.235,-,498000.0,311.1M,0.00(2.16%),5485763.0,-,-,26.67%,1.687236e+09,01/09/2021,0.00,MDCH,2.220
1,AMMB,2.90,2.9-3.02,3900000000.0,2.91,2.63-3.79,-1.27,9332600.0,9610000000.0,N/A(N/A),3181045.0,-,1.1,-2.68%,3.314011e+09,26/08/2021,0.00,AMMB,2.350
2,CIMB,4.68,4.68-4.83,16420000000.000002,4.68,2.9-4.83,0.32,26748800.0,46870000000.0,0.05(1.03%),11481116.0,14.77,1.95,36.44%,1.001418e+10,30/08/2021,14.77,CIMB,3.330
3,RHBC,5.34,5.35-5.43,5380000000.0,5.36,4.17-5.98,0.52,6028500.0,21730000000.0,0.18(3.37%),2387024.0,10.31,1.01,11.25%,4.068360e+09,01/09/2021,10.31,RHBC,6.350
4,HLCB,17.32,17.3-18.02,4680000000.0,17.48,12.34-18.7,1.96,289700.0,19640000000.0,0.36(2.07%),166766.0,8.86,1.04,37.68%,1.133789e+09,01/09/2021,8.86,HLCB,2.215
5,MBBM,8.17,8.16-8.32,22890000000.0,8.17,6.96-9.01,0.6,11938400.0,95530000000.0,0.52(6.50%),6173324.0,13.53,1.07,8.21%,1.169334e+10,26/08/2021,13.53,MBBM,10.310
6,MBSS,0.61,0.61-0.635,1640000000.0,0.61,0.485-0.77,0.06,18132000.0,4370000000.0,0.02(3.33%),6570521.0,10.23,1.16,11.93%,7.171484e+09,02/09/2021,10.23,MBSS,4.065
7,PUBM,3.94,3.93-4.13,9480000000.0,3.94,2.97-4.97,0.26,56126400.0,76480000000.0,0.03(3.30%),17805673.0,15.08,0.86,13.87%,1.941069e+10,24/08/2021,15.08,PUBM,3.430
8,BSTB,2.00,1.99-2.04,4830000000.0,2.00,1.88-2.27,0.14,811000.0,2680000000.0,0.12(6.16%),577756.0,14.73,0.08,-4.27%,1.341900e+09,16/11/2021,14.73,BSTB,14.250
9,DRBM,1.59,1.6-1.65,14550000000.0,1.60,1.58-2.25,0.41,2419300.0,3070000000.0,0.02(1.24%),1283879.0,3.85,2.26,-22.82%,1.933237e+09,26/08/2021,3.85,DRBM,1.150


,Revenue,EPS,Volume,Market Cap,Average Vol. (3m),Beta,1-Year Change,Shares Outstanding,PE,Symbol,Yield
0,490.27M,-,498000.0,311.1M,5485763.0,-,26.67%,1.687236e+09,0.00,MDCH,2.220
1,3900000000.0,-1.27,9332600.0,9610000000.0,3181045.0,1.1,-2.68%,3.314011e+09,0.00,AMMB,2.350
2,16420000000.000002,0.32,26748800.0,46870000000.0,11481116.0,1.95,36.44%,1.001418e+10,14.77,CIMB,3.330
3,5380000000.0,0.52,6028500.0,21730000000.0,2387024.0,1.01,11.25%,4.068360e+09,10.31,RHBC,6.350
4,4680000000.0,1.96,289700.0,19640000000.0,166766.0,1.04,37.68%,1.133789e+09,8.86,HLCB,2.215
5,22890000000.0,0.6,11938400.0,95530000000.0,6173324.0,1.07,8.21%,1.169334e+10,13.53,MBBM,10.310
6,1640000000.0,0.06,18132000.0,4370000000.0,6570521.0,1.16,11.93%,7.171484e+09,10.23,MBSS,4.065
7,9480000000.0,0.26,56126400.0,76480000000.0,17805673.0,0.86,13.87%,1.941069e+10,15.08,PUBM,3.430
8,4830000000.0,0.14,811000.0,2680000000.0,577756.0,0.08,-4.27%,1.341900e+09,14.73,BSTB,14.250
9,14550000000.0,0.41,2419300.0,3070000000.0,1283879.0,2.26,-22.82%,1.933237e+09,3.85,DRBM,1.150


,Revenue,EPS,Volume,Market Cap,Average Vol. (3m),Beta,1-Year Change,Shares Outstanding,PE,Symbol,Yield
0,4.902700e+08,0.00,498000.0,3.111000e+08,5485763.0,0.00,26.67,1.687236e+09,0.00,MDCH,2.220
1,3.900000e+09,-1.27,9332600.0,9.610000e+09,3181045.0,1.10,-2.68,3.314011e+09,0.00,AMMB,2.350
2,1.642000e+10,0.32,26748800.0,4.687000e+10,11481116.0,1.95,36.44,1.001418e+10,14.77,CIMB,3.330
3,5.380000e+09,0.52,6028500.0,2.173000e+10,2387024.0,1.01,11.25,4.068360e+09,10.31,RHBC,6.350
4,4.680000e+09,1.96,289700.0,1.964000e+10,166766.0,1.04,37.68,1.133789e+09,8.86,HLCB,2.215
5,2.289000e+10,0.60,11938400.0,9.553000e+10,6173324.0,1.07,8.21,1.169334e+10,13.53,MBBM,10.310
6,1.640000e+09,0.06,18132000.0,4.370000e+09,6570521.0,1.16,11.93,7.171484e+09,10.23,MBSS,4.065
7,9.480000e+09,0.26,56126400.0,7.648000e+10,17805673.0,0.86,13.87,1.941069e+10,15.08,PUBM,3.430
8,4.830000e+09,0.14,811000.0,2.680000e+09,577756.0,0.08,-4.27,1.341900e+09,14.73,BSTB,14.250
9,1.455000e+10,0.41,2419300.0,3.070000e+09,1283879.0,2.26,-22.82,1.933237e+09,3.85,DRBM,1.150


<class 'pandas.core.frame.DataFrame'>
Int64Index: 923 entries, 0 to 922
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Revenue             923 non-null    float64 
 1   EPS                 923 non-null    float64 
 2   Volume              923 non-null    float64 
 3   Market Cap          923 non-null    float64 
 4   Average Vol. (3m)   923 non-null    float64 
 5   Beta                923 non-null    float64 
 6   1-Year Change       923 non-null    float64 
 7   Shares Outstanding  923 non-null    float64 
 8   PE                  923 non-null    float64 
 9   Symbol              429 non-null    category
 10  Yield               429 non-null    float64 
dtypes: category(1), float64(10)
memory usage: 100.6 KB


None

In [20]:
standard_attribs = list(df_dividends)
#code_attrib = ["code"]

#standard_pipeline = Pipeline([
#        ('selector', DataFrameSelector(standard_attribs)),
#        ('preprocessor', DataPreprocessor()),
#        ('scaler', StandardScaler()),
#    ])

#code_pipeline = Pipeline([
#       ('selector', DataFrameSelector(code_attrib)),
#       ('code_encoder', OneHotEncoder(sparse=False))
#    ])



full_pipeline = FeatureUnion(transformer_list=[
        ("standard_attribs", standard_pipeline),
        #("code_attrib", code_pipeline)
    ])

df_prepared = full_pipeli

NameError: name 'df_dividends' is not defined